In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from code_for_hw3_part1 import * 
from tqdm.notebook import tqdm
import random
import sys
from copy import deepcopy

Importing code_for_hw03
Imported tidy_plot, plot_separator, plot_data, plot_nonlin_sep, cv, rv, y, positive, score
Datasets: super_simple_separable_through_origin(), super_simple_separable(), xor(), xor_more()
Tests for part 2: test_linear_classifier_with_features, mul, make_polynomial_feature_fun, 
                  test_with_features
Also loaded: perceptron, one_hot_internal, test_one_hot


In [2]:
import csv
rawdata = None
with open('auto-mpg.tsv', newline='') as csvfile:
    mpgreader = csv.reader(csvfile, delimiter='\t', quotechar='"')
    tsvdata = [row for row in mpgreader if len(row)]
    rawdatatitle = tsvdata[:1]
    rawdata = tsvdata[1:]
print(rawdatatitle)
print(rawdata[:10])

[['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin', 'car_name']]
[['-1', '8', '304', '193', '4732', '18.5', '70', '1', 'hi 1200d'], ['-1', '8', '307', '200', '4376', '15', '70', '1', 'chevy c20'], ['-1', '8', '360', '215', '4615', '14', '70', '1', 'ford f250'], ['-1', '8', '318', '210', '4382', '13.5', '70', '1', 'dodge d200'], ['-1', '8', '350', '180', '3664', '11', '73', '1', 'oldsmobile omega'], ['-1', '8', '400', '150', '4997', '14', '73', '1', 'chevrolet impala'], ['-1', '8', '429', '208', '4633', '11', '72', '1', 'mercury marquis'], ['-1', '8', '350', '160', '4456', '13.5', '72', '1', 'oldsmobile delta 88 royale'], ['-1', '8', '350', '180', '4499', '12.5', '73', '1', 'oldsmobile vista cruiser'], ['-1', '8', '383', '180', '4955', '11.5', '71', '1', 'dodge monaco (sw)']]


In [3]:
default_iterations = 10000
def perceptron(data, labels, params = {}):
    T = params.get('T', default_iterations)
    pbar = params.get('pbar', None)
    d = len(data)
    n = len(data[0])
    mistake = 0
    th = np.array([[0]] * d)
    th0 = np.array([[0]])
    for iter in range(T):
        if pbar is not None:
            pbar.update(1)
        for i in range(n):
            x = np.array([data[:,i]])
            y = labels[:,i]
            if np.sign(np.dot(x, th) + th0) != y:
                th0 = th0 + y
                th = th + y * x.T
                mistake = mistake + 1
    params['mistakes'] = mistake
    return (th, th0)

def eval_classifier(learner, data_train, labels_train, data_test, labels_test, params = {}):
    th, th0 = learner(data_train, labels_train, params)
    def eval(x, y):
        return 1 if np.sign(np.dot(x, th) + th0) == y else 0
    
    n = len(data_train[0])
    train_acc = np.average([eval(np.array([data_train[:,i]]), labels_train[:,i]) for i in range(n)])
    m = len(data_test[0])    
    test_acc = np.average([eval(np.array([data_test[:,i]]), labels_test[:,i]) for i in range(m)])
    params['train_acc'] = train_acc
    params['test_acc'] = test_acc
    return 1 - train_acc


def xval_classifier(learner, data, labels, fold, params = {}):
    data_chunk = np.array_split(data, fold, axis=1)
    lables_chunk = np.array_split(labels, fold, axis=1)
    perf = 0
    for i in range(fold):
        data_train = np.concatenate([data_chunk[j] for j in range(fold) if i != j], axis=1)
        lables_train = np.concatenate([lables_chunk[j] for j in range(fold) if i != j], axis=1)
        perf = perf + eval_classifier(learner, data_train, lables_train, data_chunk[i], lables_chunk[i], params)
    return perf / fold

data = np.array([[2, 3,  4,  5]])
labels = np.array([[1, 1, -1, -1]])

eval_classifier(perceptron, data, labels, data, labels)

0.0

In [32]:
def eval_feature_extraction(extractor, data):
    print(f'Evaluate {extractor.__name__}:')
    random.shuffle(data)
    data, labels = extractor(data)
    # Task 1: perceptron errors
    with tqdm(total=default_iterations * 12, file=sys.stdout) as pbar:
        print(f'\tParsed data dimension: {data.shape}')
        params = {'pbar': pbar}
        perceptron(data, labels, params)
        print(f'\tTask 1: mistakes = {params["mistakes"]}')
    # Task 2, 3: train and test errors
        train_len = int(round(len(data[0]) * .9))
        train_data, test_data = np.split(data, [train_len], axis=1)
        train_labels, test_labels = np.split(labels, [train_len], axis=1)
        params = {'pbar': pbar}
        eval_classifier(perceptron, train_data, train_labels, test_data, test_labels, params)
        print(f'\tTask 2, 3: train accuracy = {params["train_acc"]}, test accuracy = {params["test_acc"]}')
    # Task 4: cross validation
        params = {'pbar': pbar}
        avg_err = xval_classifier(perceptron, data, labels, 10, params)
    print(f'\tTask 4: Average error rate = {avg_err}')

In [33]:
from collections import OrderedDict
def standardize(row):
    mean = np.mean(row)
    stdev = np.std(row)
    row = row - mean
    row /= stdev
    return row 

def simple_mpg_feature_extraction(data):
    labels = np.array([[float(row[0]) for row in data]])
    cylinders =     standardize(np.array([[int(row[1]) for row in data]]))
    displacement =  standardize(np.array([[float(row[2]) for row in data]]))
    horsepower =    standardize(np.array([[int(row[3]) for row in data]]))
    weight =        standardize(np.array([[int(row[4]) for row in data]]))
    acceleration =  standardize(np.array([[float(row[5]) for row in data]]))
    # model_year = OrderedDict.fromkeys(row[6] for row in data])	
    # origin = dropped
    # car_name = dropped
    return np.concatenate((cylinders, displacement, horsepower, weight, acceleration), axis=0), labels

def unstd_mpg_feature_extraction(data):
    labels = np.array([[float(row[0]) for row in data]])
    cylinders =     np.array([[int(row[1]) for row in data]])
    displacement =  np.array([[float(row[2]) for row in data]])
    horsepower =    np.array([[int(row[3]) for row in data]])
    weight =        np.array([[int(row[4]) for row in data]])
    acceleration =  np.array([[float(row[5]) for row in data]])
    return np.concatenate((cylinders, displacement, horsepower, weight, acceleration), axis=0), labels

def year_one_hot_mpg_feature_extraction(data):
    cdhwa, labels = simple_mpg_feature_extraction(data)
    year_collection = sorted(set([row[6] for row in data]))
    model_year =  np.array([[0] * len(data) for i in range(len(year_collection))])
    for i, row in enumerate(data):
        model_year[year_collection.index(row[6])][i] = 1
    return np.concatenate((cdhwa, model_year), axis=0), labels

def year_thermometer_mpg_feature_extraction(data):
    cdhwa, labels = simple_mpg_feature_extraction(data)
    year_collection = sorted(set([row[6] for row in data]))
    model_year =  np.array([[0] * len(data) for i in range(len(year_collection))])
    for i, row in enumerate(data):
        for j in range(year_collection.index(row[6]) + 1):
            model_year[j][i] = 1
    return np.concatenate((cdhwa, model_year), axis=0), labels


def simple_poly2_mpg_feature_extraction(data):
    p1basis, labels = simple_mpg_feature_extraction(data)
    p2basis = deepcopy(p1basis)
    for i, ri in enumerate(p1basis):
        for j, rj in enumerate(p1basis):
            if i <= j:
               p2basis = np.append(p2basis, standardize([ri * rj]), axis=0)
    return p2basis, labels

def year_one_hot_poly2_mpg_feature_extraction(data):
    p2basis, labels = simple_poly2_mpg_feature_extraction(data)
    year_collection = sorted(set([row[6] for row in data]))
    model_year =  np.array([[0] * len(data) for i in range(len(year_collection))])
    for i, row in enumerate(data):
        model_year[year_collection.index(row[6])][i] = 1
    return np.concatenate((p2basis, model_year), axis=0), labels

eval_feature_extraction(simple_mpg_feature_extraction, deepcopy(rawdata))
eval_feature_extraction(unstd_mpg_feature_extraction, deepcopy(rawdata))
eval_feature_extraction(year_one_hot_mpg_feature_extraction, deepcopy(rawdata))
eval_feature_extraction(year_thermometer_mpg_feature_extraction, deepcopy(rawdata))
eval_feature_extraction(simple_poly2_mpg_feature_extraction, deepcopy(rawdata))
eval_feature_extraction(year_one_hot_poly2_mpg_feature_extraction, deepcopy(rawdata))

Evaluate simple_mpg_feature_extraction:


  0%|          | 0/120000 [00:00<?, ?it/s]

	Parsed data dimension: (5, 392)
	Task 1: mistakes = 647398
	Task 2, 3: train accuracy = 0.8753541076487252, test accuracy = 0.8205128205128205
	Task 4: Average error rate = 0.16072302343548803
Evaluate unstd_mpg_feature_extraction:


  0%|          | 0/120000 [00:00<?, ?it/s]

	Parsed data dimension: (5, 392)
	Task 1: mistakes = 867347
	Task 2, 3: train accuracy = 0.8328611898016998, test accuracy = 0.9743589743589743
	Task 4: Average error rate = 0.1459720254957507
Evaluate year_one_hot_mpg_feature_extraction:


...

	Parsed data dimension: (18, 392)
	Task 1: mistakes = 392588
	Task 2, 3: train accuracy = 0.8810198300283286, test accuracy = 0.8974358974358975
	Task 4: Average error rate = 0.16697382822559875
Evaluate simple_poly2_mpg_feature_extraction:


  0%|          | 0/120000 [00:00<?, ?it/s]

	Parsed data dimension: (20, 392)
	Task 1: mistakes = 468241
	Task 2, 3: train accuracy = 0.8838526912181303, test accuracy = 0.8717948717948718
	Task 4: Average error rate = 0.09864070950296162
Evaluate year_one_hot_poly2_mpg_feature_extraction:


  0%|          | 0/120000 [00:00<?, ?it/s]

	Parsed data dimension: (33, 392)
	Task 1: mistakes = 292370
	Task 2, 3: train accuracy = 0.9518413597733711, test accuracy = 0.9230769230769231
	Task 4: Average error rate = 0.06888037599793974


In [ ]:
def one_hot(x, k):
    return np.array([[1] if i == x - 1 else [0] for i in range(k)])

one_hot(3, 7)

In [ ]:
encode = np.vectorize(lambda x : one_hot(x, 6).T, signature="()->(n)")
data = encode(np.array([[2, 3,  4,  5]]))[0].T
labels = np.array([[1, 1, -1, -1]])

display(data)
perceptron(data, labels)

In [ ]:
data =   np.array([[1, 2, 3, 4, 5, 6]])
labels = np.array([[1, 1, -1, -1, 1, 1]])
data = encode(data)[0].T

display(data)
perceptron(data, labels)

In [ ]:
[x*(x-1)//2+2*x+1 for x in [1, 10, 20, 30, 40, 50]]

In [ ]:
test_with_features(super_simple_separable_through_origin, 1, False, False)
test_with_features(super_simple_separable, 1, False, False)
test_with_features(xor, 2, False, False)
test_with_features(xor_more, 3, False, False)